###  Formatting a CSV into a DataFrame¶
In the previous version of this notebook, some gymnastics were required to read my small csv into Spark. That would be much easier today using Spark 2.X than it was with 1.4, there's also the fact that Databricks makes it very simple to use their data importer to access data from Amazon's S3. I've simply used their [web GUI to create a table](https://docs.databricks.com/user-guide/tables.html) named "realestate" that can be read with Spark SQL, and from there, all the goodness of Spark dataframes is available to me.

In [2]:
df = sqlContext.sql("SELECT * FROM realestate")

Spark leverages "lazy" execution, so nothing has been executed yet. We can force spark to do some work and take a peak at the data by calling "take" on our dataframe. A take of 5 shows the first 5 rows from our original csv. Again, if you read the older version of this tutorial, you will see that this is skipping through a lot of logic for converting between RDD's and dataframes that was a real inconvenience back in the day.

In [4]:
df.take(5)

Although the dataframe responds to some RDD syntax such as 'take', the formatting is not especially readable. For a more readable view, try 'show'. Having tried take and show on some much larger data sets too, they seem to be executed differently. Show is sometimes executed in a tiny fraction of the time of take.

In [6]:
df.show(5)

## Pandas
When working in Spark with Python, there is also an easy option to convert to Pandas on the fly, assuming your dataframe is small enough that it could be held in memory. Pandas dataframes can also be converted to Spark dataframes.

In [8]:
df.toPandas().head()

## Dataframe operations¶
Once you have converted to a dataframe, many operations like selecting rows/columns and doing counts becomes much easier. Keep in mind that Spark dataframes are not mutable. For example, if we'd like a dataframe with only the homes in zip code 95815, the syntax should be very comfortable for Pandas and R users.

In [10]:
favorite_zip = df[df.zip == 95815]
favorite_zip.show(5)

It's also straightforward to choose a subset of columns from your dataframe by calling '.select' on that dataframe.

In [12]:
df.select('city','beds').show(10)

Here's a count of how many houses have different numbers of bedrooms. Note that some houses seem to have 0 beds, which should be impossible. We'll keep that in mind for later.

In [14]:
df.groupBy("beds").count().show()

Describe allows us to retrieve summary statistics on one or more columns. Results are returned in a dataframe, so .show() is necessary to see them. Not only are some of the bed values 0, but baths and square feet each have mins of zero, which are bad data points.

In [16]:
df.describe(['baths', 'beds','price','sq__ft']).show()

I'm curious to take a look at the distribution of house prices too. I see that we've got a house that sold for $1551 on the low end and one for $884,790 on the high end. Databricks notebooks do allow you to use Matplotlib. The limitation seems to be that you have to bring whatever you are plotting into memory with something like Pandas, so this wouldn't work for big data without sampling aggregating first (if you know of a way plotting directly from a Spark dataframe, please post in the comments).

In [18]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
plt.hist(df.toPandas()['price'], bins = 25)
plt.xlabel('US Dollars')
plt.title('Distribution of Sacramento Home Prices ')
display(fig)

Looks like we've got a right skewed distribution here with some outliers on the left as well. As a simple response to this for our exploratory model, later we'll remove prices below $50,000 or above $450,000.

### Regression with SparkML
As opposed to the previous iteration of this notebook where I used the RDD API for MLlib, this time I'll be using dataframes. Let's do a very simple linear regression.

In [21]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

I'll start by creating a dataframe df that has only the subset of features I'm interested in. I'm going to predict home price from the number of baths, beds, and square feet.

In [23]:
df = df.select('price','baths','beds','sq__ft')

Let's remove those rows that have suspicious 0 values for any of the features we want to use for prediction and some of the more extreme house prices to get something closer to a normal distribution

In [25]:
df = df[df.baths > 0]
df = df[df.beds > 0]
df = df[df.sq__ft > 0]
df = df[df.price >  50000]
df = df[df.price < 400000]
df.describe(['baths','beds','price','sq__ft']).show()

Instantiate a vector assembler object that will vectorize all of the feature columns that we are interested in using in our model.

In [27]:
features = ["baths","beds","sq__ft"]
assembler = VectorAssembler(
    inputCols=features,
    outputCol='features')

Now create a new dataframe with our vector assembled. Spark wants a column containing feature vectors in order to train a model.

In [29]:
assembled_df = assembler.transform(df)
assembled_df.show(5)

Perform a train/test split of our data.

In [31]:
train, test = assembled_df.randomSplit([0.6, 0.4], seed=0)

Now we can instantiate a linear regression model and train it. We'll need to designate which columns contain the features and the target.

In [33]:
lr = LinearRegression(maxIter=10).setLabelCol("price").setFeaturesCol("features")
model = lr.fit(train)

Now that our model is trained, lets evaluate on the test data.

In [35]:
testing_summary = model.evaluate(test)

Here's a look at some predictions made by the model.

In [37]:
testing_summary.predictions.select('price','baths','beds','sq__ft','prediction').show(10)

Let's also take a look at the RMSE to get an idea of how good our average prediction was.

In [39]:
testing_summary.rootMeanSquaredError

That's it for this basic introduction to Spark dataframes and MLlib. Looking back at the tutorial I wrote 2 years ago with this same content, it's amazing to see how much Spark has matured and improved. This notebook required a fraction of the previous code, and far less frustration. If you're interested in next steps, I suggest checking out the [Spark ML page on tuning models](https://spark.apache.org/docs/2.1.0/ml-tuning.html) to leverage concepts such as grid searching hyperparameters with cross validation. Also, if you are serious about running Spark against large data sets, I'd advise beginners to start thinking about when it is (and is not) to [leverage caching](http://spark.apache.org/docs/latest/quick-start.html#caching) for their use cases.